In [26]:
!pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [27]:
import pandas as pd
import pydriller
import csv
from pydriller import Repository

In [28]:
import csv
import re
import os
import subprocess
from pydriller import Repository
from git import Repo


local_repo_paths = [
    "/Users/tejasmacipad/Desktop/Third_year/STT/lab4/butterknife",
    "/Users/tejasmacipad/Desktop/Third_year/STT/lab4/retrofit",
    "/Users/tejasmacipad/Desktop/Third_year/STT/lab4/glide"
]
output_csv_filename = "diff_analysis.csv"



csv_header = [
    "repository_name", "old_file_path", "new_file_path", "commit_SHA",
    "parent_commit_SHA", "commit_message", "diff_myers", "diff_hist", "Discrepancy"
]

with open(output_csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)

    five_mb = 5 * 1024 * 1024 #to measure the growth of csv file (preventive measure so that it does not become too big to process)
    printat = five_mb

    for path in local_repo_paths: #choosing each of the repository
        print(f"Processing repository: {os.path.basename(path)}")

        try:
            for commit in Repository(path).traverse_commits(): #traversing across each of the repo
                if not commit.parents:
                    continue 

                parent_sha = commit.parents[0] #taking parent of the current commit, [0] used to avoid issues in case of merging commit

                for mod in commit.modified_files:
                    if mod.filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.zip')): #ignoring the processing on the files which are in the binary format
                         continue

                    filepathdiff = mod.new_path or mod.old_path #getting the parth of the current committed commit, we take or with mod.old_path to avoid exception in case where the file might have been deleted later on
                    if not filepathdiff: 
                        continue 

                    common_args = ["git", "-C", path, "diff", parent_sha, commit.hash, "--", filepathdiff] #this is the parser or arg to the command to find the differences for all the valid and filtered commits.
                    
                    # Build args with the algorithm flag BEFORE the "--"
                    myers_args = [
                        "git", "-C", path, "diff", "--diff-algorithm=myers", "--no-color",
                        parent_sha, commit.hash, "--", filepathdiff
                    ]
                    hist_args = [
                        "git", "-C", path, "diff", "--diff-algorithm=histogram", "--no-color",
                        parent_sha, commit.hash, "--", filepathdiff
                    ]

                    myers_proc = subprocess.run(myers_args, capture_output=True, text=True, encoding="utf-8", errors="ignore")
                    hist_proc  = subprocess.run(hist_args,  capture_output=True, text=True, encoding="utf-8", errors="ignore")

                    myers_diff_text = myers_proc.stdout #diff according to the myers algorithm
                    hist_diff_text = hist_proc.stdout #diff according to the hist_diff_text algorithm

                    norm_myers = "\n".join([re.sub(r'\s+', '', line) for line in myers_diff_text.splitlines() if re.sub(r'\s+', '', line)]) #normalization as mentioned in the question
                    norm_hist = "\n".join([re.sub(r'\s+', '', line) for line in hist_diff_text.splitlines() if re.sub(r'\s+', '', line)])

                    discrepancy_found = "Yes" if norm_myers != norm_hist else "No" # manual check, if they exactly overal then no discrepancy which is the majority case, else "Yes"
                    
                    csv_writer.writerow([ #writing the extracted content to the csv file
                        os.path.basename(path), mod.old_path, mod.new_path,
                        commit.hash, parent_sha, commit.msg,
                        myers_diff_text, hist_diff_text, discrepancy_found
                    ])

                    current_size = os.path.getsize(output_csv_filename)
                    if current_size >= printat:
                        size_in_mb = current_size / (1024 * 1024)
                        print(f"--> CSV file size has reached {size_in_mb:.2f} MB.")
                        printat += five_mb 

        except Exception as error:
            print(f"An error  processing {os.path.basename(path)}: {error}")
            continue

print(f"\nAnalysis finished. Results are in {output_csv_filename}")
df = pd.read_csv("diff_analysis.csv")
row = df["Discrepancy"]
yes_count = (row.str.strip().str.lower() == "yes").sum()
print(f"Number of 'Yes' in Discrepancy: {yes_count}")

Processing repository: butterknife
--> CSV file size has reached 5.01 MB.
--> CSV file size has reached 10.00 MB.
--> CSV file size has reached 15.01 MB.
Processing repository: retrofit
--> CSV file size has reached 20.01 MB.
--> CSV file size has reached 25.01 MB.
--> CSV file size has reached 30.01 MB.
--> CSV file size has reached 35.00 MB.
--> CSV file size has reached 40.01 MB.
--> CSV file size has reached 45.03 MB.
--> CSV file size has reached 50.01 MB.
--> CSV file size has reached 55.00 MB.
--> CSV file size has reached 60.03 MB.
Processing repository: glide
--> CSV file size has reached 65.02 MB.
--> CSV file size has reached 70.01 MB.
--> CSV file size has reached 75.00 MB.
--> CSV file size has reached 80.01 MB.
--> CSV file size has reached 85.01 MB.
--> CSV file size has reached 90.00 MB.
--> CSV file size has reached 95.00 MB.
--> CSV file size has reached 100.00 MB.
--> CSV file size has reached 105.01 MB.
--> CSV file size has reached 110.01 MB.
--> CSV file size has 

In [30]:
import pandas as pd


df = pd.read_csv("diff_analysis.csv")

def categorize_file(path):
    if pd.isna(path):
        return "Other"
    path = path.lower()
    
    if "test" in path:
        return "Test code"
    
    source_exts = (".py", ".java", ".c", ".cpp", ".h", ".hpp", ".js", ".ts", ".go", ".rb", ".swift")
    if path.endswith(source_exts):
        return "Source code"
    
    if "readme" in path:
        return "README"
    
    if "license" in path:
        return "LICENSE"
    
    return "Other"

df["file_category"] = df["new_file_path"].fillna(df["old_file_path"]).apply(categorize_file)

summary = (
    df.groupby("file_category")
    .agg(
        Total_Files=("file_category", "size"),
        Mismatches=("Discrepancy", lambda x: (x == "Yes").sum())
    )
)

summary["Mismatch_%"] = (summary["Mismatches"] / summary["Total_Files"] * 100).round(5)

print(summary)

               Total_Files  Mismatches  Mismatch_%
file_category                                     
LICENSE                 17           0     0.00000
Other                 7149          52     0.72737
README                 381          12     3.14961
Source code          14515         616     4.24389
Test code             9282         428     4.61108


In [1]:
!git clone git@github.com:TejasLohia21/TejasLohia1.4.git


Cloning into 'TejasLohia1.4'...
